In [1]:
%load_ext cython

We check numerically that $e\in \mathbb{S}^{n-1}$ and $A\in \mathbb{R}^{n\times n}$ be arbitrary.
Then
$$
  |\mathbb{S}^{n-1}|^{-1}
  \int_{\mathbb{S}^{n-1}}
  \left|e^T s\right|^{p-2}
  s^T\left[(p-1)I  + (2-p)ee^T\right]As
  \,\mathrm{d}s = K_{n,p} \mathrm{trace} A,
$$
where
$$
    K_{p,n}=
  |\mathbb{S}^{n-1}|^{-1}
  \int_{\mathbb{S}^{n-1}}
  \left|e^T s\right|^{p}
  \,\mathrm{d}s.
$$

In [2]:
%%cython -f -c=-O3 -c=-Wno-deprecated-declarations -c=-Wno-unreachable-code-fallthrough
#-a

from cpython.pycapsule cimport (PyCapsule_New,
                                PyCapsule_GetPointer)
from cpython.mem cimport PyMem_Malloc,  PyMem_Free
from libc.math cimport sin, cos, pow, abs
from libc.stdlib cimport malloc, free
import scipy
from cython cimport boundscheck, wraparound, cdivision, nonecheck

@boundscheck(False)
@wraparound(False)
@nonecheck(False)
@cdivision(True)
cdef double sph_jac(int n, double* phi):
    """Jacobian of the spherical coordinates"""
    cdef int i
    jac = 1.0
    for i in range(n-2):
        jac *= pow(sin(phi[i]),n-2-i)
    return jac

@boundscheck(False)
@wraparound(False)
@nonecheck(False)
@cdivision(True)
cdef void sph2cart(int n, double* phi, double* z):
    """Convert spherical to cartezian coordinates"""
    cdef double sin_prod = 1.0
    cdef int i
    for i in range(n-1):
        z[i] = sin_prod*cos(phi[i])
        sin_prod *= sin(phi[i])
    z[n-1] = sin_prod

@boundscheck(False)
@wraparound(False)
@nonecheck(False)
@cdivision(True)
cdef double integrand_main(int N, double* phi, void* user_data):
    """The integrand, written in Cython"""
    # Extract the value of p, vector e, and matrix A
    # Cython uses array access syntax for pointer dereferencing!
    cdef int     n = N+1
    cdef double  p = (<double*>user_data)[0]
    cdef double* e = (<double*>user_data)+1
    cdef double* A = (<double*>user_data)+1+n
    cdef double* z = (<double*>user_data)+1+n+n*n
    cdef int i, j
    cdef double e_dot_z, e_dot_z_abs, z_A_z, e_A_z, A_z_i
    #
    # first, we need to transform from spherical to Cartesian coordinates
    sph2cart(n, phi, z)
    # then we can evaluate the integrand
    e_dot_z = 0.0
    z_A_z   = 0.0
    e_A_z   = 0.0
    for i in range(n):
        A_z_i = 0.0
        for j in range(n):
            A_z_i += A[i*n+j]*z[j]
        e_dot_z += e[i]*z[i]
        z_A_z   += z[i]*A_z_i
        e_A_z   += e[i]*A_z_i
    e_dot_z_abs = abs(e_dot_z)
    if (p>=2) or (e_dot_z_abs>1.0E-12):
        return pow(e_dot_z_abs,p-2)*((p-1)*z_A_z + (2-p)*e_dot_z*e_A_z) * sph_jac(n, phi)
    else:
        return 0.0

#
# Pack numpy arrays containing the parameters of the integrand
#
cdef object pack_user_data(int n, double p, double[:] e, double[:,:] A):
    """Wrap data in a PyCapsule for transport."""
    # Allocate memory
    cdef double* data_ptr = <double*> PyMem_Malloc(sizeof(double)*(1+n+n*n+n))
    data_ptr[0] = p
    cdef int i,j
    for i in range(n):
        data_ptr[1+i] = e[i]
    for i in range(n):
        for j in range (n):
            data_ptr[1+n+i*n+j] = A[i,j]
    # the last n doubles is a buffer
    return PyCapsule_New(<void*>data_ptr, NULL, free_user_data)

cdef void free_user_data(capsule):
    """Free the memory our value is using up."""
    PyMem_Free(PyCapsule_GetPointer(capsule, NULL))

    
def get_low_level_callable_main(int n, double p, double[:] e, double[:,:] A):
    # scipy.LowLevelCallable expects the function signature to
    # appear as the "name" of the capsule
    func_capsule = PyCapsule_New(<void*>integrand_main,
                                 "double (int, double *, void *)",
                                 NULL)
    data_capsule = pack_user_data(n, p, e, A)
    
    return scipy.LowLevelCallable(func_capsule, data_capsule)

In [3]:
import numpy as np
from scipy import integrate, special

In [4]:
def compute_integrals(n,p,e,A):
    ranges = [[0,np.pi]]*(n-2)
    ranges.append([0,2*np.pi])
    # compute the area of the sphere first
    Sn  = 2*pow(np.pi,n/2) / special.gamma(n/2)
    # compute Kpn
    Kpn = pow(np.pi,-0.5) * special.gamma(n/2) \
          * special.gamma((p+1)/2) / special.gamma((n+p)/2)    
    # the main integral
    integrand_main = get_low_level_callable_main(n,p,e,A)
    I, abserr      = integrate.nquad(integrand_main, ranges)
    return I/Sn/Kpn

In [5]:
%%time

p = 1.75
n = 4

rng = np.random.default_rng()

A = rng.normal(size=[n,n])
v = rng.normal(size=[n])
e = v/np.linalg.norm(v)

I = compute_integrals(n,p,e,A)


print("Integral over the sphere, I = %f" % I)
print("Trace of A,           tr(A) = %f" % np.trace(A))

Integral over the sphere, I = 1.300119
Trace of A,           tr(A) = 1.300119
CPU times: user 19.7 s, sys: 301 ms, total: 20 s
Wall time: 20 s
